Step 1: Import Python Libraries into the Jupyter Notebook.

Note: Please download the relevant Python packages to execute this notebook.

In [1]:
import pandas as pd
from keras.models import load_model
import numpy as np
import time

Using TensorFlow backend.


Step 2: Load the precalculated featurizations for each of the 1,710 drugs in the DrugBank database (SSP, c-ADMET and c-MolDes); load the DDI categorical description to explain the meaning of each type of DDI; and load the description of DB_ID to generic drug name. DB_ID refers to the DrugBank ID that is used to unequivocally refer to a specific drug structure in the DrugBank database.

In [2]:
SSP=pd.read_csv("3_1_1_Input Features for DDI MLP\DDI_MLP_SSP.csv")
c_ADMET=pd.read_csv("3_1_1_Input Features for DDI MLP\DDI_MLP_Standardized_c-ADMET.csv")
c_MolDes=pd.read_csv("3_1_1_Input Features for DDI MLP\DDI_MLP_Standardized_c-MolDes.csv")
DDI_Type_general_description=pd.read_csv("3_1_2_Misc Input Files for DDI MLP\DrugBank_DDI_Categorical_Description.csv")

DB_ID_to_Generic_Name_df=pd.read_csv("3_1_2_Misc Input Files for DDI MLP\Drugs_DB_ID_to_Generic_Name.csv")
DB_ID=list(DB_ID_to_Generic_Name_df["DB_ID"].values)
Generic_Name=list(DB_ID_to_Generic_Name_df["Drug_Generic_Name"].values)
DB_ID_to_Generic_Name={}
for i in range(len(DB_ID)):
    DB_ID_to_Generic_Name[DB_ID[i]]=Generic_Name[i]

Step 3: Load DDI MLP model into the notebook.


Note: You can safely ignore any tensorflow warnings that pop up when executing this cell.

In [3]:
start_time = time.time()
saved_model = load_model("3_1_2_Misc Input Files for DDI MLP\Optimized_DDI_MLP-NN_Arch_4_1024-Triple_Feat.h5")
model_load_time=time.time() - start_time

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


Step 4: Define a function that takes Drug A and Drug B (in the DB_ID (DrugBank ID) format within the database (1,710 drugs)), which will then create the DDI MLP input by looking up the featurization values for both drugs in Step 2, and output the expected DDI between Drug A and B (order-sensitive).

In [4]:
def execute_DDI_MLP(Drug_A, Drug_B):
    start_time = time.time()
    Drug_Pair=[Drug_A, Drug_B]
    Drug_query=[]

    for Drug in Drug_Pair:
        SSP_query=list(SSP.loc[SSP['DB_ID'] == Drug].values[0])[1:]
        c_ADMET_query=list(c_ADMET.loc[c_ADMET['DB_ID'] == Drug].values[0])[1:]
        c_MolDes_query=list(c_MolDes.loc[c_MolDes['DB_ID'] == Drug].values[0])[1:]
        temp_query=SSP_query+c_MolDes_query+c_ADMET_query 
        Drug_query=Drug_query+temp_query

    input_to_DDI_MLP=pd.DataFrame(Drug_query).T

    #Get DDI MLP Prediction:
    output_vector = saved_model.predict(input_to_DDI_MLP)
    predicted_DDI = np.argmax(output_vector, axis=1)+1
    print("Model Name: DDI MLP, [4,1024] network architecture, triple features")
    print("\n--- Query Details ---")
    print("Drug A :", Drug_A, "( Common Name:", DB_ID_to_Generic_Name[Drug_A],")")
    print("Drug B :", Drug_B, "( Common Name:", DB_ID_to_Generic_Name[Drug_B],")")
    print("\n--- Start of Query Results ---")
    print("DDI MLP predicts type", predicted_DDI, "DDI between Drug A (", DB_ID_to_Generic_Name[Drug_A], ") and B (", DB_ID_to_Generic_Name[Drug_B],").")
    print("Type", predicted_DDI, "DDI means :", list(DDI_Type_general_description.loc[DDI_Type_general_description['DDI type'] == predicted_DDI[0]].values[0])[1])
    print("--- End of Query Results ---")
    print("\nModel Load time (only loaded once per notebook start-up or reset): %.5f seconds" % (model_load_time))
    print("\nQuery Run time: %.5f seconds" % (time.time() - start_time))
    return predicted_DDI[0]

Step 5: Input any pair of drugs in the DB_ID (DrugBank ID) format within the database (1,710 drugs). DDI MLP will return its predicted DDI between the drug pair.


For example, you could try setting Drug A to "DB09000" (or Cyamemazine) and Drug B to "DB00460" (or Verteporfin). This drug pair combination is the first row DDI entry in the "DDI_MLP_Test_DB_Pair_and_Interaction.xlsx" file. Cyamemazine may increase the photosensitizing activities of Verteporfin (Type 75 DDI). For other DDI pairings, please refer to "DDI_MLP_Training_DB_Pair_and_Interaction.xlsx", "DDI_MLP_Validation_DB_Pair_and_Interaction.xlsx" or "DDI_MLP_Test_DB_Pair_and_Interaction.xlsx" file.

In [5]:
Drug_A="DB09000"
Drug_B="DB00460"

predicted_DDI=execute_DDI_MLP(Drug_A, Drug_B)

Model Name: DDI MLP, [4,1024] network architecture, triple features

--- Query Details ---
Drug A : DB09000 ( Common Name: Cyamemazine )
Drug B : DB00460 ( Common Name: Verteporfin )

--- Start of Query Results ---
DDI MLP predicts type [75] DDI between Drug A ( Cyamemazine ) and B ( Verteporfin ).
Type [75] DDI means : Drug A may increase the photosensitizing activities of Drug B.
--- End of Query Results ---

Model Load time (only loaded once per notebook start-up or reset): 5.26868 seconds

Query Run time: 0.42137 seconds
